In [59]:
import numpy as np
import random

from tests import generate_regression_data, test_regression_model

In [60]:
class Regression(object):
    '''Класс для предсказания действительно-значного выхода по входу - вектору из R^n.
    Используется линейная регрессия, то есть если вход X \in R^n, вектор весов W \in R^{n+1},
    то значение регрессии - это [X' 1] * W, то есть y = x1*w1 + x2*w2 + xn*wn + wn+1.
    Обучение - подгонка весов W - будет вестись на парах (x, y).

    Параметры
    ----------
    sgd : объект класса SGD
    trainiterator: объект класса TrainIterator
    n_epoch : количество эпох обучения (default = 1)
    batch_size : размер пакета для шага SGD (default = 16)
    '''

    def __init__(self, sgd, trainiterator, n_epoch=1, batch_size=16):
        self.sgd = sgd
        self.trainiterator = trainiterator
        self.n_epoch = n_epoch
        self.batch_size = batch_size
        self.W = None

    def fit(self, X, y):
        '''Обучение модели.

        Параметры
        ----------
        X : двумерный массив признаков размера n_samples x n_features
        y : массив/список правильных значений размера n_samples

        Выход
        -------
        Метод обучает веса W
        '''
        random_index = random.randint(0, len(X) - 1)
        self.W = X[random_index]

        for subX, suby in self.trainiterator:
            self.W = self.sgd.step(subX, suby, self.W)

        return self

    def predict(self, X):
        """ Предсказание выходного значения для входных векторов

        Параметры
        ----------
        X : двумерный массив признаков размера n_samples x n_features

        Выход
        -------
        y : Массив размера n_samples
        """
        y_pred = list()
        y_pred = np.dot(X, self.W)

        return y_pred

    def score(self, y_gt, y_pred):
        """Возвращает точность регрессии в виде (1 - u/v),
        где u - суммарный квадрат расхождения y_gt с y_pred,
        v - суммарный квадрат расхождения y_gt с матожиданием y_gt

        Параметры
        ----------
        y_gt : массив/список правильных значений размера n_samples
        y_pred : массив/список предсказанных значений размера n_samples

        Выход
        -------
        accuracy - точность регрессии
        """
        u, v, sum, N = 0, 0, 0, len(y_gt)
        My_gt = y_gt.mean()

        for i in range(N):
            u += (y_gt[i] - y_pred[i]) ** 2
            v += (y_gt[i] - My_gt) ** 2
            
        if N == 1:
            v = 1

        accuracy = 1 - u / v

        return accuracy

In [61]:
class SGD(object):
    '''Класс для реализации метода стохастического градиентного спуска.

    Параметры
    ----------
    grad : функция вычисления градиента
    alpha : градиентный шаг (default = 1.)

    '''

    def __init__(self, grad, alpha=1.):
        self.grad = grad
        self.alpha = alpha

    def step(self, X, y, W):
        '''Один шаг градиентного спуска.

        Параметры
        ----------
        X : двумерный массив признаков размера n_samples x n_features
        y : массив/список правильных значений размера n_samples
        W : массив весов размера n_weights

        Выход
        -------
        Метод возвращает обновленные веса
        '''
        W_new = W - self.alpha * self.grad.grad(X, y, W) / len(X)

        return W_new

In [62]:
class Grad(object):
    '''Класс для вычисления градиента по весам от функции потерь.

    Параметры
    ----------
    loss : функция потерь
    delta : параметр численного дифференцирования (default = 0.000001)
    '''

    def __init__(self, loss, delta=0.000001):
        self.loss = loss
        self.delta = delta

    def grad(self, X, y, W):
        '''Вычисление градиента.

        Параметры
        ----------
        X : двумерный массив признаков размера n_samples x n_features
        y : массив/список правильных значений размера n_samples
        W : массив весов размера n_weights

        Выход
        -------
        Метод возвращает градиент по весам W в точках X от функции потерь
        '''
        loss_gradient = np.zeros(len(W))
        for i in range(len(loss_gradient)):
            e = np.zeros(len(loss_gradient))
            e[i] = 1
            loss = self.loss.val(X, y, W)
            loss_delta = self.loss.val(X, y, W + self.delta * e)
            loss_gradient[i] = (loss_delta - loss) / self.delta

        return loss_gradient

In [63]:
class Loss(object):
    '''Класс для вычисления функции потерь.

    Параметры
    ----------
    l1_coef : коэффициент l1 регуляризации (default = 0)
    l2_coef : коэффициент l2 регуляризации (default = 0)
    '''

    def __init__(self, l1_coef=0, l2_coef=0):
        self.l1_coef = l1_coef
        self.l2_coef = l2_coef

    def val(self, X, y, W):
        '''Вычисление функции потерь.

        Параметры
        ----------
        X : двумерный массив признаков размера n_samples x n_features
        y : массив/список правильных значений размера n_samples
        W : массив весов размера n_weights

        Выход
        -------
        Метод возвращает значение функции потерь в точках X
        '''
        loss, mse, cost = 0, 0, 0

        for i in range(len(X)):
            mse += (np.dot(W, X[i]) - y[i]) ** 2

        for i in range(len(W)):
            cost += self.l1_coef * abs(W[i]) + self.l2_coef * (W[i]) ** 2

        loss += mse + cost
        return loss

In [64]:
class TrainIterator(object):
    '''Класс итератора для работы с обучающими данными.

    Параметры
    ----------
    X : двумерный массив признаков размера n_samples x n_features
    y : массив/список правильных значений размера n_samples
    n_epoch : количество эпох обучения (default = 1)
    batch_size : размер пакета для шага SGD (default = 16)
    '''

    def __init__(self, X, y, n_epoch=1, batch_size=16):
        self.X = X
        self.y = y
        self.n_epoch = n_epoch
        self.batch_size = batch_size
        self.counter = -1

    def __iter__(self):
        '''Нужно для использования итератора в цикле for
        Здесь ничего менять не надо
        '''
        return self

    def __next__(self):
        '''Выдача следующего батча.

        Выход
        -------
        Метод возвращает очередной батч как из X, так и из y
        '''
        n_samples = len(self.X)
        n_batches = (self.n_epoch * n_samples) // self.batch_size
        self.counter += 1
        if self.counter < n_batches:
            i = self.counter * self.batch_size - (n_samples * ((self.counter * self.batch_size) // n_samples))
            j = i + batch_size
            k = j - n_samples if j > n_samples else -n_samples
            return np.append(self.X[i:j], self.X[:k], axis=0), np.append(self.y[i:j], self.y[:k])
        else:
            raise StopIteration

В поисках лучшей точности для Nfeat=100, Mtrain=150, Mtest=150 основными параметрами были n_epoch и batch_size, так как остальные параметры довольны хрупкие и любое изменение в количестве нулей сильно влияет на ответ: например, при 10 эпохах и batch_size = 2 при alpha = 0.001, delta = 0.000001, l1_coef = 0.001, l2_coef = 0.001 точность была 0.58,
             а при alpha = 0.01, delta = 0.0000001, l1_coef = 0.0001, l2_coef = 0.0001 была 0.87.
Поэтому на остальных запусках использовал alpha = 0.01, delta = 0.0000001, l1_coef = 0.0001, l2_coef = 0.0001.
С 50 < n_epoch < 100 при идеальном подборе остальных параметров (batch_size так, что бы было не менее 2 батчей за эпоху) точность менялась от 0.6 до 0.9. На batch_size = 2 эпохи показывали лучшую точность (0.91 при 15 эпохах, 0.97 при 30 эпохах, 0.98 при 60 эпохах) и это очевидно, так как итераций обновления весов происходит больше и веса обучаются лучше.
На n_epoch = 100 с batch_size = 50 до batch_size = 5 (меньше не делал, так как работало долго) точность c 0.8 улучшалась до 0.98 соответсвенно.
На n_epoch = 150 с batch_size = 5 точность улучшилась до 0.99.
Увеличение эпох далее не помогало, так как на n_epoch = 200 с batch_size = 5 точность вышла 0.98.
Таким образом по умолчанию я выбрал n_epoch = 60, batch_size = 2, так они показали наилучшую точность при наименьшем времени

In [97]:
n_epoch = 60
batch_size = 2
alpha = 0.01
delta = 0.0000001
l1_coef = 0.0001
l2_coef = 0.0001

trainX, trainY, testX, testY = generate_regression_data(Nfeat=100, Mtrain=150, Mtest=150)

trainiterator = TrainIterator(trainX, trainY, n_epoch, batch_size)
loss = Loss(l1_coef, l2_coef)
grad = Grad(loss, delta)
sgd = SGD(grad, alpha)

reg = Regression(sgd, trainiterator, n_epoch, batch_size)

test_regression_model(reg, trainX, trainY, testX, testY)

TEST REGRESSION MODEL: Your accuracy is 0.986970312514349
